In [3]:
import xarray as xr
import xesmf as xe
from xmip.preprocessing import combined_preprocessing
import PyCO2SYS as pyco2 # i have to install this package through terminal for it to work
import intake
import matplotlib.pyplot as plt
import numpy as np
import warnings
import gsw
from xmip.postprocessing import combine_datasets
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import matplotlib.path as mpath
from xmip.postprocessing import combine_datasets
from xmip.postprocessing import merge_variables
from dask.diagnostics import ProgressBar
from xgcm import Grid
from xhistogram.xarray import histogram
import pandas as pd
import scipy
import cftime

In [4]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

def pick_first_member(ds_list, **kwargs):
    return ds_list[0]

In [5]:
variables = ['dissic', 'talk', 'sos', 'tos', 'fgco2', 'ph', 'co3sataragos', 'co3os']
z_kwargs = {'consolidated': True, 'use_cftime': True}
query = dict(experiment_id=['historical'], 
             table_id=['Omon'], 
             variable_id=variables,
             member_id = ['r1i1p1f1'],
             grid_label=['gr'],
              source_id=['GFDL-ESM4']
            )


cat = col.search(**query)

# print(cat.df['source_id'].unique())
dd = cat.to_dataset_dict(zarr_kwargs=z_kwargs, storage_options={'token': 'anon'},
                                preprocess=combined_preprocessing, aggregate=False)

dd = merge_variables(dd)
# list(dd_new.keys())

# temp={}
# for name,item in dd.items():
#     #print(name)
#     #print(item.data_vars)
#     present = item.data_vars
#     if all(i in present for i in variables):
#         #print(name)
#         temp[name]=item
# dd = temp


# dd_gn = combine_datasets(
#     dd_gn,
#     pick_first_member,
#     match_attrs=['source_id', 'grid_label', 'experiment_id', 'table_id']
# )

list(dd.items())
# dd_new_new_gn


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


/tmp/ipykernel_1078/680137587.py:15: DeprecationWarning: cdf_kwargs and zarr_kwargs are deprecated and will be removed in a future version. Please use xarray_open_kwargs instead.
  dd = cat.to_dataset_dict(zarr_kwargs=z_kwargs, storage_options={'token': 'anon'},


/srv/conda/envs/notebook/lib/python3.10/site-packages/xmip/preprocessing.py:106: UserWarning: Renaming failed with 'co3os'
  warnings.warn(f"Renaming failed with {e}")
/srv/conda/envs/notebook/lib/python3.10/site-packages/xmip/preprocessing.py:106: UserWarning: Renaming failed with 'sos'
  warnings.warn(f"Renaming failed with {e}")
/srv/conda/envs/notebook/lib/python3.10/site-packages/xmip/preprocessing.py:106: UserWarning: Renaming failed with 'fgco2'
  warnings.warn(f"Renaming failed with {e}")
/srv/conda/envs/notebook/lib/python3.10/site-packages/xmip/preprocessing.py:106: UserWarning: Renaming failed with 'ph'
  warnings.warn(f"Renaming failed with {e}")
/srv/conda/envs/notebook/lib/python3.10/site-packages/xmip/preprocessing.py:106: UserWarning: Renaming failed with 'co3sataragos'
  warnings.warn(f"Renaming failed with {e}")
/srv/conda/envs/notebook/lib/python3.10/site-packages/xmip/preprocessing.py:106: UserWarning: Renaming failed with 'dissic'
  warnings.warn(f"Renaming failed 

[('GFDL-ESM4.gr.historical.Omon.r1i1p1f1',
  <xarray.Dataset>
  Dimensions:         (y: 180, bnds: 2, x: 360, time: 1980, member_id: 1,
                       dcpp_init_year: 1, vertex: 4, lev: 35)
  Coordinates: (12/14)
    * y               (y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
      lat_bounds      (y, bnds, x) float64 dask.array<chunksize=(180, 2, 360), meta=np.ndarray>
    * x               (x) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
      lon_bounds      (x, bnds, y) float64 dask.array<chunksize=(360, 2, 180), meta=np.ndarray>
    * time            (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
      time_bounds     (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
      ...              ...
      lon_verticies   (vertex, x, y) float64 dask.array<chunksize=(1, 360, 180), meta=np.ndarray>
      lat_verticies   (vertex, x, y) float64 dask.array<chunksize=(1, 360, 180), meta=np.ndarray>
    * member_id       (member

In [6]:
ds = dd['GFDL-ESM4.gr.historical.Omon.r1i1p1f1']

In [8]:
ds.isel(time = slice(0,240)).time

<xarray.DataArray 'time' (time: 240)>
array([cftime.DatetimeNoLeap(1850, 1, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 2, 15, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 3, 16, 12, 0, 0, 0, has_year_zero=True),
       ...,
       cftime.DatetimeNoLeap(1869, 10, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1869, 11, 16, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1869, 12, 16, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 1850-01-16 12:00:00 ... 1869-12-16 12:00:00
Attributes:
    axis:           T
    bounds:         time_bnds
    calendar_type:  noleap
    description:    Temporal mean
    long_name:      time
    standard_name:  time

# Time Slice: Jan 1850 to Dec 1869

In [9]:
ds = ds.isel(time = slice(0, 240))

# Select only first depth level (z = 2.5 m) for relevant variables

In [10]:
ds = ds.isel(lev = 0)

# Density from `gsw`

In [11]:

ds['p'] = gsw.p_from_z(-1*ds['lev'], ds['y'], geo_strf_dyn_height=0, sea_surface_geopotential=0)*xr.ones_like(ds['talk']) # expand along time dimension

ds['sigma0'] = gsw.sigma0(ds['sos'],ds['tos'])
ds

<xarray.Dataset>
Dimensions:         (y: 180, bnds: 2, x: 360, time: 240, member_id: 1,
                     dcpp_init_year: 1, vertex: 4)
Coordinates: (12/14)
  * y               (y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bounds      (y, bnds, x) float64 dask.array<chunksize=(180, 2, 360), meta=np.ndarray>
  * x               (x) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    lon_bounds      (x, bnds, y) float64 dask.array<chunksize=(360, 2, 180), meta=np.ndarray>
  * time            (time) object 1850-01-16 12:00:00 ... 1869-12-16 12:00:00
    time_bounds     (time, bnds) object dask.array<chunksize=(240, 2), meta=np.ndarray>
    ...              ...
    lon_verticies   (vertex, x, y) float64 dask.array<chunksize=(1, 360, 180), meta=np.ndarray>
    lat_verticies   (vertex, x, y) float64 dask.array<chunksize=(1, 360, 180), meta=np.ndarray>
  * member_id       (member_id) object 'r1i1p1f1'
  * dcpp_init_year  (dcpp_init_year) float64 nan
    lev             float64 2.5
    lev_bounds      (bnds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
Dimensions without coordinates: bnds, vertex
Data variables:
    sos             (member_id, dcpp_init_year, time, y, x) float32 dask.array<chunksize=(1, 1, 120, 180, 360), meta=np.ndarray>
    fgco2           (member_id, dcpp_init_year, time, y, x) float32 dask.array<chunksize=(1, 1, 120, 180, 360), meta=np.ndarray>
    co3os           (member_id, dcpp_init_year, time, y, x) float32 dask.array<chunksize=(1, 1, 240, 180, 360), meta=np.ndarray>
    talk            (member_id, dcpp_init_year, time, y, x) float32 dask.array<chunksize=(1, 1, 12, 180, 360), meta=np.ndarray>
    co3sataragos    (member_id, dcpp_init_year, time, y, x) float32 dask.array<chunksize=(1, 1, 240, 180, 360), meta=np.ndarray>
    tos             (member_id, dcpp_init_year, time, y, x) float32 dask.array<chunksize=(1, 1, 120, 180, 360), meta=np.ndarray>
    ph              (member_id, dcpp_init_year, time, y, x) float32 dask.array<chunksize=(1, 1, 16, 180, 360), meta=np.ndarray>
    dissic          (member_id, dcpp_init_year, time, y, x) float32 dask.array<chunksize=(1, 1, 12, 180, 360), meta=np.ndarray>
    p               (y, member_id, dcpp_init_year, time, x) float64 dask.array<chunksize=(180, 1, 1, 12, 360), meta=np.ndarray>
    sigma0          (member_id, dcpp_init_year, time, y, x) float64 dask.array<chunksize=(1, 1, 120, 180, 360), meta=np.ndarray>
Attributes: (12/49)
    Conventions:                      CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            36500.0
    comment:                          <null ref>
    ...                               ...
    intake_esm_attrs:experiment_id:   historical
    intake_esm_attrs:member_id:       r1i1p1f1
    intake_esm_attrs:table_id:        Omon
    intake_esm_attrs:grid_label:      gr
    intake_esm_attrs:version:         20190726
    intake_esm_attrs:_data_format_:   zarr

In [12]:
ds = ds.squeeze()
ds

<xarray.Dataset>
Dimensions:         (y: 180, bnds: 2, x: 360, time: 240, vertex: 4)
Coordinates: (12/14)
  * y               (y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bounds      (y, bnds, x) float64 dask.array<chunksize=(180, 2, 360), meta=np.ndarray>
  * x               (x) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    lon_bounds      (x, bnds, y) float64 dask.array<chunksize=(360, 2, 180), meta=np.ndarray>
  * time            (time) object 1850-01-16 12:00:00 ... 1869-12-16 12:00:00
    time_bounds     (time, bnds) object dask.array<chunksize=(240, 2), meta=np.ndarray>
    ...              ...
    lon_verticies   (vertex, x, y) float64 dask.array<chunksize=(1, 360, 180), meta=np.ndarray>
    lat_verticies   (vertex, x, y) float64 dask.array<chunksize=(1, 360, 180), meta=np.ndarray>
    member_id       <U8 'r1i1p1f1'
    dcpp_init_year  float64 nan
    lev             float64 2.5
    lev_bounds      (bnds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
Dimensions without coordinates: bnds, vertex
Data variables:
    sos             (time, y, x) float32 dask.array<chunksize=(120, 180, 360), meta=np.ndarray>
    fgco2           (time, y, x) float32 dask.array<chunksize=(120, 180, 360), meta=np.ndarray>
    co3os           (time, y, x) float32 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    talk            (time, y, x) float32 dask.array<chunksize=(12, 180, 360), meta=np.ndarray>
    co3sataragos    (time, y, x) float32 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    tos             (time, y, x) float32 dask.array<chunksize=(120, 180, 360), meta=np.ndarray>
    ph              (time, y, x) float32 dask.array<chunksize=(16, 180, 360), meta=np.ndarray>
    dissic          (time, y, x) float32 dask.array<chunksize=(12, 180, 360), meta=np.ndarray>
    p               (y, time, x) float64 dask.array<chunksize=(180, 12, 360), meta=np.ndarray>
    sigma0          (time, y, x) float64 dask.array<chunksize=(120, 180, 360), meta=np.ndarray>
Attributes: (12/49)
    Conventions:                      CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            36500.0
    comment:                          <null ref>
    ...                               ...
    intake_esm_attrs:experiment_id:   historical
    intake_esm_attrs:member_id:       r1i1p1f1
    intake_esm_attrs:table_id:        Omon
    intake_esm_attrs:grid_label:      gr
    intake_esm_attrs:version:         20190726
    intake_esm_attrs:_data_format_:   zarr

# Conversion

DIC and TA are given in mol/m3. `PyCO2SYS` takes units of umol/kg. 

$$ \frac{mol}{m^3} \times \frac{10^6 \ umol}{mol} \times \frac{1 \ m^3}{1000 + \sigma \ kg} = \frac{1e6}{1000 + \sigma}$$

In [13]:
conversion =  1e6/(1000+ds.sigma0)
results = pyco2.sys(par1=ds['talk']*conversion, par2 = ds['dissic']*conversion, par1_type=1, par2_type = 2, temperature=ds['tos'], salinity = ds['sos'])

# Remove the meaningless dimensions

In [14]:
ds = ds.isel(bnds = 0, vertex = 0)

# Add calculated fugacity to the dataset

In [15]:
ds['fugacity'] = xr.Variable(dims = ['time','y', 'x'], data = results['fCO2'])
ds['omega'] = xr.Variable(dims = ['time','y', 'x'], data = results['saturation_aragonite'])

In [16]:
ds['fugacity'].attrs = {"units": 'uatm', 
                        'description': 'CO2 fugacity, calculated with PyCO2SYS. DIC and TA converted from mol/m3 to umol/kg using gsw-derived density'}
ds['omega'].attrs = {"units": '', 
                        'description': 'aragonite saturation state, calculated with PyCO2SYS. DIC and TA converted from mol/m3 to umol/kg using gsw-derived density'}
ds.omega

<xarray.DataArray 'omega' (time: 240, y: 180, x: 360)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [1.90899654, 1.90888347, 1.90866381, ..., 1.90980488,
         1.90952724, 1.90930557],
        [1.93494942, 1.93465661, 1.93440033, ..., 1.93553606,
         1.93533243, 1.93515505],
        [1.94715696, 1.94688843, 1.94662457, ..., 1.94801247,
         1.9477103 , 1.94743022]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [1.89604882, 1.8960639 , 1.89639459, ..., 1.89648667,
         1.89622972, 1.89610752],
        [1.87994617, 1.88000502, 1.87991612, ..., 1.8797022 ,
         1.87988659, 1.87990257],
        [1.85448272, 1.85441293, 1.85437413, ..., 1.85454045,
         1.85459448, 1.85454189]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [1.87259148, 1.8729362 , 1.87342061, ..., 1.87153803,
         1.87191655, 1.8722322 ],
        [1.86434841, 1.86428817, 1.86417069, ..., 1.86464781,
         1.8645859 , 1.86446102],
        [1.83639157, 1.83609241, 1.83583778, ..., 1.83715099,
         1.83698033, 1.83668822]]])
Coordinates: (12/14)
  * y               (y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bounds      (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
  * x               (x) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    lon_bounds      (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
  * time            (time) object 1850-01-16 12:00:00 ... 1869-12-16 12:00:00
    time_bounds     (time) object dask.array<chunksize=(240,), meta=np.ndarray>
    ...              ...
    lon_verticies   (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    lat_verticies   (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    member_id       <U8 'r1i1p1f1'
    dcpp_init_year  float64 nan
    lev             float64 2.5
    lev_bounds      float64 dask.array<chunksize=(), meta=np.ndarray>
Attributes:
    units:        
    description:  aragonite saturation state, calculated with PyCO2SYS. DIC a...

In [17]:
ds = ds.to_netcdf('GFDLESM4historical_1850_1870.nc')

In [18]:
da = xr.open_mfdataset('GFDLESM4historical_1850_1870.nc')

In [19]:
da

<xarray.Dataset>
Dimensions:         (y: 180, x: 360, time: 240)
Coordinates: (12/14)
  * y               (y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bounds      (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
  * x               (x) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    lon_bounds      (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
  * time            (time) object 1850-01-16 12:00:00 ... 1869-12-16 12:00:00
    time_bounds     (time) object dask.array<chunksize=(240,), meta=np.ndarray>
    ...              ...
    lon_verticies   (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    lat_verticies   (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    member_id       object ...
    dcpp_init_year  float64 ...
    lev             float64 ...
    lev_bounds      float64 ...
Data variables:
    sos             (time, y, x) float32 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    fgco2           (time, y, x) float32 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    co3os           (time, y, x) float32 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    talk            (time, y, x) float32 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    co3sataragos    (time, y, x) float32 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    tos             (time, y, x) float32 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    ph              (time, y, x) float32 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    dissic          (time, y, x) float32 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    p               (y, time, x) float64 dask.array<chunksize=(180, 240, 360), meta=np.ndarray>
    sigma0          (time, y, x) float64 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    fugacity        (time, y, x) float64 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
    omega           (time, y, x) float64 dask.array<chunksize=(240, 180, 360), meta=np.ndarray>
Attributes: (12/49)
    Conventions:                      CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            36500.0
    comment:                          <null ref>
    ...                               ...
    intake_esm_attrs:experiment_id:   historical
    intake_esm_attrs:member_id:       r1i1p1f1
    intake_esm_attrs:table_id:        Omon
    intake_esm_attrs:grid_label:      gr
    intake_esm_attrs:version:         20190726
    intake_esm_attrs:_data_format_:   zarr